In [ ]:
import pandas as pd
import geopandas as gpd
from IPython.display import display
import os
import re
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.spatial import cKDTree
mpl.rcParams['figure.dpi']= 300
DATA_DIR = os.path.join('data', 'argentina')

In [ ]:
#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension 
!jupyter nbextension enable --py --sys-prefix gmaps
import gmaps
import gmaps.datasets
with open('google_maps_key.txt', 'r') as fapi:
    gmaps.configure(api_key=fapi.read().strip())

In [ ]:
from utils.utils import normalize_dpto_name, validate_dpto_indexes

# Get all databases

In [ ]:
import generate_databases
generate_databases.store_all()

# pxdpto geo test

In [ ]:
PXLOC = os.path.join(DATA_DIR, 'indec', 'pxdptodatosok.shp')
geodata = gpd.read_file(PXLOC, encoding='utf-8')
from utils.utils import normalize_dpto_name, validate_dpto_indexes
geodata['departamen'] = [normalize_dpto_name(n) for n in geodata['departamen']]
geodata['link'] = [int(n) for n in geodata['link']]
geodata

In [ ]:
geodata[geodata['provincia']=='Salta']

In [ ]:
geodata[geodata['provincia']=='Santa Fe'].groupby('departamen').mean()

In [ ]:
geodata['geometry'][1]

In [ ]:
print(geodata['geometry'][1].centroid)

In [ ]:
# Store our latitude and longitude
puntos = [[dpto.centroid.y, dpto.centroid.x] for dpto in geodata['geometry']]
latitudes = [dpto.centroid.y for dpto in geodata['geometry']]
longitudes = [dpto.centroid.x for dpto in geodata['geometry']]
pesos = geodata['hogares']

m = gmaps.Map()
m.add_layer(gmaps.heatmap_layer(
    puntos, weights=pesos,
    max_intensity=float(max(pesos)), point_radius=10.0
))
m

# Densidad

In [ ]:
DENS_FILE = os.path.join(DATA_DIR, 'datosgobar-densidad-poblacion', 'pais.geojson')
db_densidad = gpd.read_file(DENS_FILE, encoding='utf-8')
db_densidad['area'] = pd.to_numeric(db_densidad['area'])
db_densidad['poblacion'] = pd.to_numeric(db_densidad['poblacion'])
db_densidad['viviendas'] = pd.to_numeric(db_densidad['viviendas'])
db_densidad['hogares'] = db_densidad['hogares'].apply(lambda x: int(re.sub(r'(\d+).0+', r'\1', x)) if x else 0)
print(db_densidad.crs)
db_densidad

In [ ]:
db_densidad['poblacion'].iplot(kind="histogram", bins=100, theme="white", title="Histograma de poblacion de secciones del pais", xTitle='Poblacion', yTitle='Cant.')

In [ ]:
np.sum(db_densidad['poblacion'])

In [ ]:
f"population: {np.sum(db_densidad['poblacion'])/1000:.0f}k"

In [ ]:
_ = list(map(display, random.choices(db_densidad['geometry'], k=3)))

In [ ]:
db_densidad["area"].iplot(kind="histogram", bins=10000, theme="white", title="Histograma de areas de secciones del pais", xTitle='Area', yTitle='Cant.')

In [ ]:
db_densidad["densidad"].iplot(kind="histogram", bins=10000, theme="white", title="Histograma de densidades de secciones del pais", xTitle='Densidad', yTitle='Cant.')

In [ ]:
diff=(set(map(int, db_densidad['dpto_id']))^set(map(int, geodata['link'])))
diff

In [ ]:
print("Missing deparments in density")
geodata.loc[geodata['link'].isin(diff)][['link', 'departamen', 'provincia']]

In [ ]:
ax = db_densidad[db_densidad['dpto_id']=='82084'].plot(column='densidad', cmap='hot')
ax.set_title('Densidad de secciones de Rosario')
ax.set_xlim(-60.7, -60.6)
ax.set_ylim(-33.0, -32.9)

In [ ]:
ax = db_densidad[db_densidad['prov_id']=='82'].plot(column='densidad', cmap='hot')
ax.set_title('Densidad de secciones de Santa Fe')
ax

In [ ]:
# Store our latitude and longitude
puntos = [[dpto.centroid.y, dpto.centroid.x] for dpto in db_densidad['geometry']]
pesos = db_densidad['hogares']

m = gmaps.Map()
m.add_layer(gmaps.heatmap_layer(
    puntos, weights=pesos,
    max_intensity=float(max(pesos)), point_radius=5.0
))
m

# pxlocdatos

In [ ]:
db_densidad.to_crs(epsg=4326,inplace=True)

In [ ]:
densidad_datos = gpd.read_file(
    os.path.join(DATA_DIR, 'datosgobar-densidad-poblacion', 'datos.geojson'), encoding='utf-8')
densidad_datos

In [ ]:
pxlocdatos = gpd.read_file(os.path.join(DATA_DIR, 'indec', 'pxlocdatos.shp'), encoding='utf-8')
# pxlocdatos['departamen'] = [normalize_dpto_name(n) for n in pxlocdatos['departamen']]
# pxlocdatos['link'] = [int(n) for n in pxlocdatos['link']]
pxlocdatos

In [ ]:
pxlocdatos.crs = db_densidad.crs

In [ ]:
joined = gpd.tools.sjoin(pxlocdatos,db_densidad,how="left")
joined

# Distancias entre secciones

In [ ]:
#https://www.eye4software.com/hydromagic/documentation/supported-map-grids/Argentina
# proyeccion para argentina
db_densidad.to_crs(epsg=5349,inplace=True)

In [ ]:
santafe = db_densidad[db_densidad['dpto_id']=='82063']['geometry'].iloc[0].centroid
rosario = db_densidad[db_densidad['dpto_id']=='82084']['geometry'].iloc[0].centroid
f"{rosario.distance(santafe)/1000:.0f} km"

In [ ]:
upper_bound=3000
centroids = np.array(list(zip(db_densidad.geometry.centroid.x, db_densidad.geometry.centroid.y)))
btree = cKDTree(centroids)
dist, idx = btree.query(centroids, k=2000, distance_upper_bound=upper_bound)

In [ ]:
nearests_count = pd.DataFrame([sum(1 for d in plist if d>1e-9 and d<upper_bound) for plist in dist])
nearests_count.iplot(kind="histogram", bins=100, theme="white", title=f"Histograma de cantidad de secciones vecinas a menos de {upper_bound/1000:.0f} km", xTitle='Cantidad de zonas cercanas', yTitle='Cant. de zonas con esa cantidad de zonas cercanas')

In [ ]:
sum(nearests_count)

# Ministerio educacion database

In [ ]:
pd.options.display.max_columns = None
SCHOOL_HDF = os.path.join(DATA_DIR, 'ministerio-educacion', 'matricula_y_secciones.hdf')
schooldb = pd.read_hdf(SCHOOL_HDF, 'matricula_y_secciones')
schooldb = schooldb.replace(to_replace="Ciudad de Buenos Aires", value="Ciudad Autónoma de Buenos Aires")
schooldb

In [ ]:
count_cols = list(filter(lambda s: s.startswith('Alumnos con Sobreedad') or s.startswith('Repitentes') or s.startswith('Matrícula.'), schooldb.columns))

In [ ]:
schooldb['total_alumnos'] = schooldb.loc[:,count_cols].sum(axis=1)

In [ ]:
pd.options.display.max_columns = None
SCHOOL_HDF = os.path.join(DATA_DIR, 'ministerio-educacion', 'matricula_por_edad.hdf')
schooldb = pd.read_hdf(SCHOOL_HDF, 'matricula_por_edad')
schooldb

In [ ]:
count_cols = list(filter(lambda s: 'años' in s, schooldb.columns))

In [ ]:
schooldb['total_alumnos'] = schooldb[count_cols].sum(axis=1)

In [ ]:
graph = schooldb[schooldb['Ámbito'] == 'Rural']['total_alumnos']
print(f"Escuelas rurales {len(graph)}")
graph.iplot(kind="histogram", bins=1000, theme="white", title="Histograma de cantidad de alumnos por escuela, rurales", xTitle='Cantidad de alumnos', yTitle='Cant.')

In [ ]:
graph = schooldb[(schooldb['Ámbito'] == 'Urbano') & (schooldb['total_alumnos']<800)]['total_alumnos']
print(f"Escuelas urbanas {len(graph)}")
graph.iplot(kind="histogram", bins=1000, theme="white", title="Histograma de cantidad de alumnos por escuela, urbanas", xTitle='Cantidad de alumnos', yTitle='Cant.')

# TDMA
Transito medio diario argentina

In [ ]:
tdma = gpd.read_file(os.path.join(DATA_DIR, 'transporte', 'tdma2017.geojson'), encoding='utf-8')
tdma

In [ ]:
ax = tdma.plot(column='valor', cmap='hot')
ax.set_title('TDMA Argentina')
#ax.set_xlim(-60.7, -60.6)
#ax.set_ylim(-33.0, -32.9)

# Buses interburbano